In [ ]:
#new ipynb

In [1]:
!git clone https://github.com/neheller/kits23
%cd kits23
!pip3 install -e .

Cloning into 'kits23'...
remote: Enumerating objects: 15613, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 15613 (delta 42), reused 62 (delta 30), pack-reused 15526 (from 1)
Receiving objects: 100% (15613/15613), 493.94 MiB | 43.06 MiB/s, done.
Resolving deltas: 100% (14011/14011), done.
Updating files: 100% (5720/5720), done.
/content/kits23
Obtaining file:///content/kits23
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/deepmind/surface-distance.git to /tmp/pip-install-7avuh4dc/surface-distance_c87575428c014946a2b7eabf15983b12
  Running command git clone --filter=blob:none --quiet https://github.com/deepmind/surface-distance.git /tmp/pip-install-7avuh4dc/surface-distance_c87575428c014946a2b7eabf15983b12
  Resolved https://github.com/deepmind/surface-distance.git to commit 1f805cea446805e559badc80dd27f34175e4173d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!kits23_download_data


Found 489 cases to download

Dowloading case_00588...: 100% 489/489 [09:11<00:00,  1.13s/it]


In [2]:
# 1. Restart runtime after installing if prompted by Colab
!pip install -q "torch" torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q monai
!pip install -q nibabel scikit-image pandas tqdm SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 42.9 MB/s eta 0:00:00


In [5]:
import os, json, glob
import random

root = "/content/kits23/dataset"

# Get all case folders
case_folders = sorted([d for d in glob.glob(os.path.join(root, "case_*")) if os.path.isdir(d)])

datalist = {"training": []}

num_folds = 5
random.seed(42)  # for reproducibility

for case_path in case_folders:
    case_id = os.path.basename(case_path)

    # Paths
    label_path = os.path.join(case_path, "segmentation.nii.gz")
    image_path = os.path.join(case_path, "imaging.nii.gz")

    if not os.path.exists(label_path) or not os.path.exists(image_path):
        print(f"Skipping {case_id}, missing image or label")
        continue

    # Random fold assignment
    fold = random.randint(0, num_folds - 1)

    datalist["training"].append({
        "image": os.path.relpath(image_path, start=root),
        "label": os.path.relpath(label_path, start=root),
        "image_filename": os.path.basename(image_path),
        "fold": fold
    })

dataset_json = {
    "name": "KiTS23_custom_folds",
    "description": "KiTS23 formatted dataset with random 5-fold split",
    "reference": "",
    "licence": "",
    "release": "0.0",
    "modality": {"0": "CT"},
    "labels": {"0": "background", "1": "kidney_and_mass", "2":"mass", "3":"tumor"},
    "numTraining": len(datalist["training"]),
    "training": datalist["training"],
    "test": []
}

with open(os.path.join(root, "dataset.json"), "w") as f:
    json.dump(dataset_json, f, indent=2)

print("Wrote dataset.json with", len(datalist["training"]), "cases and random fold assignments")


Wrote dataset.json with 489 cases and random fold assignments


In [6]:
pip install fire

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 4.9 MB/s eta 0:00:00


In [14]:
!sudo apt-get install python3.10 python3.10-dev python3.10-distutils
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'python3-distutils' instead of 'python3.10-distutils'
python3-distutils is already the newest version (3.10.8-1~22.04).
python3-distutils set to manually installed.
python3.10 is already the newest version (3.10.12-1~22.04.11).
python3.10 set to manually installed.
python3.10-dev is already the newest version (3.10.12-1~22.04.11).
python3.10-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [12]:
!pip install monai==1.3.0 nibabel torch torchvision einops transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.8 MB/s eta 0:00:00
  Attempting uninstall: monai
    Found existing installation: monai 1.5.1
    Uninstalling monai-1.5.1:
      Successfully uninstalled monai-1.5.1


In [13]:
import os, glob
from monai.data import Dataset, DataLoader
from monai.transforms import (
    LoadImaged, EnsureChannelFirstd, Spacingd, ScaleIntensityRanged,
    CropForegroundd, Orientationd, ToTensord, Compose
)
from sklearn.model_selection import train_test_split

root = "/content/kits23/dataset"
cases = sorted(glob.glob(os.path.join(root, "case_*")))

data_dicts = [{"image": os.path.join(c, "imaging.nii.gz"), "label": os.path.join(c, "segmentation.nii.gz")} for c in cases]
train_files, val_files = train_test_split(data_dicts, test_size=0.2, random_state=42)

train_transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image", "label"]),
    Orientationd(keys=["image", "label"], axcodes="RAS"),
    Spacingd(keys=["image", "label"], pixdim=(1.5, 1.5, 2.0), mode=("bilinear", "nearest")),
    ScaleIntensityRanged(keys=["image"], a_min=-200, a_max=200, b_min=0.0, b_max=1.0, clip=True),
    CropForegroundd(keys=["image", "label"], source_key="image"),
    ToTensord(keys=["image", "label"])
])

val_transforms = train_transforms

train_ds = Dataset(data=train_files, transform=train_transforms)
val_ds = Dataset(data=val_files, transform=val_transforms)

train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=2)


AttributeError: 'FileFinder' object has no attribute 'find_module'